# Imports

In [ ]:
# Python version: 3.11.0
# matplotlib==3.10.0
# pandas==2.2.3
# numpy==2.2.1
# scipy==1.15.0
# scikit-learn==1.6.1
# seaborn==0.13.2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Graph Stats

In [ ]:
graph_stat_files = {"./graph_hdagg_metis.txt", "./graph_florida_stats.txt", "./graph_erdos_renyi_stats.txt", "./graph_band_stats.txt", "./graph_iChol_stats.txt"}

In [ ]:
# Reading files into pandas dataframe
graph_df = pd.concat( [pd.read_csv(file) for file in graph_stat_files ], ignore_index=True )

In [ ]:
# Dropping duplicates
graph_df.drop_duplicates(inplace = True)

In [ ]:
# Changing index to Graph
graph_df = graph_df.set_index("Graph", drop=True)

In [ ]:
# Adding double precision floating point operations to the dataframe
graph_df["FLOP_double_precision"] = 2 * graph_df["Edges"] + graph_df["Vertices"]

In [ ]:
# Adding number of non-zeroes to the dataframe
graph_df["Number_of_non-zeroes"] = graph_df["Edges"] + graph_df["Vertices"]

## Graph Filter

In [ ]:
# Graph filter templates

# All graphs
all_graphs = set(graph_df.index)

# Erdos-Renyi graphs
erdos_renyi_graphs = set( [g for g in all_graphs if g[:5] == "Erdos" ] )

# Random Bandwidth graphs
random_band_graphs = set( [g for g in all_graphs if g[:10] == "RandomBand" ] )

# Florida post Eigen Cholesky graphs
cholesky_graphs = set( [g for g in all_graphs if g[-9:] == "_postChol" ] )
florida_metis_graphs = set( [g for g in all_graphs if g[-6:] == "_metis" ] )

# Florida graphs
florida_graphs = set( [g for g in all_graphs if g not in random_band_graphs and g not in erdos_renyi_graphs and g[:4] != "inst"  and g not in florida_metis_graphs and g not in cholesky_graphs])


In [ ]:
# Setting graph filters for subsequent SpTrSV data analysis
graph_subset = all_graphs 
# graph_subset = erdos_renyi_graphs
# graph_subset = random_band_graphs
# graph_subset = florida_graphs

# SpTrSV Data

In [ ]:
# Folder location
folder_location = "./SpTrSV_Data/plot_22_reorder/"

In [ ]:
# Listing all files in folder_location
data_files = set([file for file in os.listdir(folder_location)])
data_files = [file for file in data_files if file[:3] != "log"]

In [ ]:
# Specifying Datatypes
data_type_dic = {
    "Graph":                            "object",
    "Machine":                          "object",
    "Algorithm":                        "object",
    "Permutation":                      "object",
    "SpTrSV_Runtime":                  "float64",
    "Work_Cost":                         "int64",
    "Base_Comm_Cost":                    "int64",
    "Supersteps":                        "int64",
    "_Base_Buffered_Sending":            "int64",
    "Base_CostsTotalCommunication":    "float64",
    "Schedule_Compute_time":             "int64",
    "Processors":                        "int64",
    "BSP_g":                             "int64",
    "BSP_l":                             "int64",
    "Scheduling_Threads":                "int64",
}

data_default_na_val_dic = {
    "Graph":                              "",
    "Machine":                            "",
    "Algorithm":                          "",
    "Permutation":                        "",
    "SpTrSV_Runtime":                  "0.0",
    "Work_Cost":                         "0",
    "Base_Comm_Cost":                    "0",
    "Supersteps":                        "1",
    "_Base_Buffered_Sending":            "0",
    "Base_CostsTotalCommunication":    "0.0",
    "Schedule_Compute_time":             "1",
    "Processors":                        "0",
    "BSP_g":                             "0",
    "BSP_l":                             "0",
    "Scheduling_Threads":                "1",
}

In [ ]:
# Reading files into pandas dataframe
SpTrSV_df = pd.concat( [pd.read_csv( folder_location + file) for file in data_files], ignore_index=True )

In [ ]:
# Data Deleting folder structure from 'Graph' column
SpTrSV_df["Graph"] = SpTrSV_df["Graph"].str.split("/").str[-1]

In [ ]:
# Adding BSP parameters to the dataframe
SpTrSV_df[["Processors", "BSP_g", "BSP_l"]] = SpTrSV_df["Machine"].str.split("_", n=2, expand=True).reindex(range(3), axis=1)
SpTrSV_df["Processors"] = SpTrSV_df["Processors"].astype("object").str.slice(start=1).astype("int64", errors="ignore")
SpTrSV_df["BSP_g"] = SpTrSV_df["BSP_g"].astype("object").str.slice(start=1).astype("int64", errors="ignore")
SpTrSV_df["BSP_l"] = SpTrSV_df["BSP_l"].astype("object").str.slice(start=1).astype("int64", errors="ignore")

In [ ]:
# Casting to correct Datatypes
for key, val in data_type_dic.items():
    SpTrSV_df[key] = SpTrSV_df[key].fillna(data_default_na_val_dic[key]).astype(val)

In [ ]:
# Function to compute giga double precision floating point operations, denoted by GFP64
def compute_GFP64(time, graph, df = graph_df):
    flop = df.at[graph ,"FLOP_double_precision"]
    return (flop / time) / 1000000000

In [ ]:
# Adding giga double precision floating point operations, denoted by GFP64, to the dataframe
SpTrSV_df["GFP64/s"] = SpTrSV_df[["Graph", "SpTrSV_Runtime"]].apply(lambda x: compute_GFP64(x["SpTrSV_Runtime"], x["Graph"]), axis=1)

In [ ]:
# Set schedule compute time at least 1ms
SpTrSV_df["Schedule_Compute_time"] = SpTrSV_df["Schedule_Compute_time"].replace(0,1)

In [ ]:
# Data sorting
SpTrSV_df.sort_values([ "Graph", "Algorithm" ], axis=0, inplace=True)

## Filters (incl. Algorithm, Processor)

In [ ]:
# Setting algorithm filter
alg_filter_set = set(SpTrSV_df["Algorithm"].unique())
alg_filter_set = set(["SMGreedyBspGrowLocalAutoCoresParallel"])
alg_filter_set.add("Serial")

In [ ]:
# Setting processor filter
proc_filter = {22}

In [ ]:
perm_filter_set = SpTrSV_df["Permutation"].unique()

perm_filter_set = set(['NO_PERMUTE', 'LOOP_PROCESSORS'])
SpTrSV_df_filtered = SpTrSV_df[ SpTrSV_df["Permutation"].isin(perm_filter_set) ]
#perm_filter_set
print(SpTrSV_df_filtered["Permutation"].unique())

In [ ]:
# Applying algorithm filter
SpTrSV_df_filtered = SpTrSV_df_filtered[ SpTrSV_df_filtered["Algorithm"].isin(alg_filter_set) ]

In [ ]:
# Applying graph filter
SpTrSV_df_filtered = SpTrSV_df_filtered[ SpTrSV_df_filtered["Graph"].isin(graph_subset) ]


In [ ]:
# Applying processor filter
SpTrSV_df_filtered = SpTrSV_df_filtered[ SpTrSV_df_filtered["Processors"].isin(proc_filter) ]

## Evaluation

In [ ]:
# Setting up new pandas dataframe with geometric mean of GFP64 and speed-up over serial execution

geom_mean_FLOPS_df = pd.DataFrame(columns=["Processors", "Graphs", "Algorithm", "Permutation","GFP64/s", "Speedup_over_Serial", "Speedup_over_HDagg" , "Schedule_Compute_time", "Schedule_Compute_time_speedup_over_HDagg", "Supersteps_relative_to_Wavefront", "Supersteps_relative_to_HDagg"])

for name, group in SpTrSV_df_filtered.groupby(["Processors", "Graph"]):
    serial_flops = np.exp( np.log(group[ group["Algorithm"] == "Serial" ]["GFP64/s"]).mean() )
    HDagg_flops = np.exp( np.log(group[ group["Algorithm"] == "HDagg_original" ]["GFP64/s"]).mean() )
    HDagg_schedule_compute_time = group[ group["Algorithm"] == "HDagg_original" ]["Schedule_Compute_time"].mean()
    HDagg_supersteps = group[ group["Algorithm"] == "HDagg_original" ]["Supersteps"].mean()
  
    for alg, alg_group in group.groupby(["Algorithm", "Permutation"]):
        flops = np.exp( np.log(alg_group["GFP64/s"]).mean() )
        alg_schedule_compute_time = alg_group["Schedule_Compute_time"].mean()
        alg_supersteps = alg_group["Supersteps"].mean()
        wavefront_supersteps = graph_df.at[name[1], "Longest_Path"]
        temp_df = pd.DataFrame( [[name[0], name[1], alg[0], alg[1], flops, flops/serial_flops, flops/HDagg_flops, alg_schedule_compute_time, HDagg_schedule_compute_time/alg_schedule_compute_time, alg_supersteps/wavefront_supersteps, alg_supersteps/HDagg_supersteps]], columns=["Processors", "Graphs", "Algorithm", "Permutation", "GFP64", "Speedup_over_Serial", "Speedup_over_HDagg","Schedule_Compute_time", "Schedule_Compute_time_speedup_over_HDagg", "Supersteps_relative_to_Wavefront", "Supersteps_relative_to_HDagg"] )
        geom_mean_FLOPS_df = pd.concat([geom_mean_FLOPS_df, temp_df], ignore_index=True)

In [ ]:
# Adding logarithm of speed-up over serial and HDagg
geom_mean_FLOPS_df["Log2_speedup_over_Serial"] = np.log2( geom_mean_FLOPS_df["Speedup_over_Serial"] )

### GFP64/s

In [ ]:
# Average Log speed-ups over Serial
geom_mean_FLOPS_df[["Processors", "Algorithm", "Permutation","Log2_speedup_over_Serial"]].groupby(["Processors","Algorithm", "Permutation"]).mean()

florida_FLOPS = geom_mean_FLOPS_df[geom_mean_FLOPS_df["Graphs"].isin(florida_graphs)]
florida_agg = florida_FLOPS[["Processors", "Algorithm","Permutation","Log2_speedup_over_Serial"]].groupby(["Processors","Algorithm","Permutation"]).mean()
florida_agg["Geommean"] = np.exp2(florida_agg["Log2_speedup_over_Serial"])

###########################################################
#####  The output of this cell corresponds to Table 7.3
###########################################################

print(np.around(florida_agg["Geommean"], decimals=2))